In [ ]:
# Import necessary libraries
import cv2
import numpy as np
import os
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.preprocessing import StandardScaler
from tqdm import tqdm
import joblib
from tkinter import filedialog
from IPython.display import display
from PIL import Image

In [ ]:
# Helper function to apply CLAHE in Lab color space and extract features
def extract_features(image_path):
    """Extract color features from an image using Lab color space and CLAHE."""
    img = cv2.imread(image_path)
    if img is None:
        return None
    # Convert to Lab color space
    lab = cv2.cvtColor(img, cv2.COLOR_BGR2LAB)
    # Apply CLAHE to the L channel
    clahe = cv2.createCLAHE(clipLimit=3.0, tileGridSize=(8, 8))
    lab[:, :, 0] = clahe.apply(lab[:, :, 0])
    # Calculate color features (mean and std) from Lab channels
    l_mean, a_mean, b_mean = lab[:, :, 0].mean(), lab[:, :, 1].mean(), lab[:, :, 2].mean()
    l_std, a_std, b_std = lab[:, :, 0].std(), lab[:, :, 1].std(), lab[:, :, 2].std()
    return [l_mean, a_mean, b_mean, l_std, a_std, b_std]

# Function to load dataset
def load_dataset(base_dir='images'):
    """Load dataset from the specified directory."""
    X, y = [], []
    # Define folder structure with labels
    classes = {
        'inRange/normal': 1,
        'inRange/boundary/light': 1,
        'inRange/boundary/dark': 1,
        'outRange/light': 0,
        'outRange/dark': 0
    }
    for folder, label in classes.items():
        folder_path = os.path.join(base_dir, folder)
        for file in tqdm(os.listdir(folder_path), desc=f"Loading {folder}"):
            img_path = os.path.join(folder_path, file)
            features = extract_features(img_path)
            if features:
                X.append(features)
                y.append(label)
    return np.array(X), np.array(y)

In [ ]:
# Load and preprocess dataset
X, y = load_dataset()

# Train/test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Normalize the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Train the model
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train_scaled, y_train)

In [ ]:
# Evaluate the model
y_pred = model.predict(X_test_scaled)
print(f"Model Accuracy: {accuracy_score(y_test, y_pred)*100:.2f}%")
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))

# Save the model and scaler for later inference
joblib.dump(model, 'graphite_walnut_model.pkl')
joblib.dump(scaler, 'graphite_walnut_scaler.pkl')

print("\nTraining complete. Model and scaler saved.")

In [ ]:
# --- Model Inference Section ---

def upload_image():
    """ Open a file dialog to upload an image """
    file_path = filedialog.askopenfilename(title="Select Image", filetypes=[("Image files", "*.jpg;*.jpeg;*.png")])
    return file_path

def validate_sample():
    print("Upload the known in-range sample:")
    ref_path = upload_image()
    ref_features = extract_features(ref_path)
    if ref_features is None:
        print("Invalid reference image. Please try again.")
        return
    
    print("Upload the sample to be validated:")
    sample_path = upload_image()
    sample_features = extract_features(sample_path)
    if sample_features is None:
        print("Invalid sample image. Please try again.")
        return
    
    # Normalize both feature vectors
    scaler = joblib.load('graphite_walnut_scaler.pkl')
    model = joblib.load('graphite_walnut_model.pkl')
    
    # Adjust sample features relative to the known reference
    normalized_sample = np.array(sample_features) - np.array(ref_features)
    normalized_sample = scaler.transform([normalized_sample])
    
    prediction = model.predict(normalized_sample)[0]
    result = "In Range" if prediction == 1 else "Out of Range"
    
    print(f"Sample Validation Result: {result}")

# Run the validation function
validate_sample()